In [1]:
using CSV, DataFrames, Dates, HiGHS, JuMP

In [2]:
df = CSV.read("data.csv", DataFrame)
D_t = df[!, "FR_load_forecast_entsoe_transparency"]

println(size(D_t))
println(first(D_t, 5))

(8760,)
[56250.0, 54300.0, 53600.0, 50000.0, 47100.0]


In [3]:
file_path = "capacity_pv.csv"
df = CSV.File(file_path; header=true) |> DataFrame
capacity_pv = Array(df)
println(first(capacity_pv, 10))
    
file_path = "capacity_wind.csv"
df = CSV.File(file_path; header=true) |> DataFrame
capacity_wind = Array(df)
println(first(capacity_wind, 10))

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.001, 0.067, 0.218]
[0.12594, 0.123693, 0.124232, 0.13045, 0.140045, 0.136766, 0.129185, 0.135791, 0.152821, 0.14136]


In [4]:
# Constants and Data
hours_per_year = 8760
lambda = 5000  
cost_of_debt = 0.04
cost_of_equity = 0.07
corporate_tax = 0.30
economic_life = 20

# A=[1299.44,634.83,459.12,437.52,812.53,237.66]

capacity = Dict(
    "Coal" => 1,
    "CCGT" => 1,
    "OCGT" => 1,
    "Onshore Wind" => capacity_wind,
    "Offshore Wind" => capacity_wind,
    "PV" => capacity_pv
)

# Technologies data
technologies = Dict(
    "Coal" => (capex=2000, om=0.03, debt_ratio=0.7, heat_rate=2.4, EA=1299.44, price =5),
    "CCGT" => (capex=950, om=0.03, debt_ratio=0.7, heat_rate=1.62, EA=634.83, price =30),
    "OCGT" => (capex=700, om=0.03, debt_ratio=0.8, heat_rate=2.5, EA=459.12, price =30),
    "Onshore Wind" => (capex=700, om=0.03, debt_ratio=0.7, heat_rate=0, EA=437.52, price =0),
    "Offshore Wind" => (capex=1300, om=0.03, debt_ratio=0.7, heat_rate=0, EA=812.53, price =0),
    "PV" => (capex=400, om=0.03, debt_ratio=0.8, heat_rate=0, EA=237.66, price =0)
)


model = Model(HiGHS.Optimizer)

@variable(model, k[g in keys(technologies)] >= 0)  # Installed capacity
@variable(model, p[g in keys(technologies), t=1:hours_per_year] >= 0)  # Hourly production
@variable(model, 0<=d[t=1:hours_per_year])




# @objective(model, Min, sum(
#     (sum(technologies[g].price * technologies[g].heat_rate * p[g,t] + (technologies[g].om * technologies[g].capex * k[g] / economic_life) + k[g]*technologies[g].EA - D_t[t]*lambda) 
#     for t in 1:hours_per_year) for g in keys(technologies)
# ))

@objective(model,Min,sum((technologies[g].om * technologies[g].capex + technologies[g].EA )*k[g]+sum(technologies[g].heat_rate *technologies[g].price*p[g,i]/1000 for i in 1:hours_per_year)  for g in keys(technologies))-sum(lambda*d[i] for i in 1:hours_per_year))


@constraint(model, [t=1:hours_per_year], sum(p[g, t] for g in keys(technologies)) == d[t])
@constraint(model,[i in 1:hours_per_year],d[i] <= D_t[i])


# New constraint on capacity
for g in ["Coal", "CCGT", "OCGT"]
    @constraint(model, [t=1:hours_per_year], p[g, t] <= k[g])
end
for t in 1:hours_per_year
    @constraint(model, p["Onshore Wind", t] <= capacity_wind[t]*k["Onshore Wind"])
    @constraint(model, p["Offshore Wind", t] <= capacity_wind[t]*k["Offshore Wind"])
    @constraint(model, p["PV", t] <= capacity_pv[t]*k["PV"])
end

#print(model)

optimize!(model)

println("\n")
println("Point I")
println("Optimal capacities (kW):")
total_production = 0
production_by_tech = Dict(g => 0.0 for g in keys(technologies))
for g in keys(technologies)
    cap = value(k[g])  # This ensures 'cap' remains a floating-point number
    println("$(g): $(cap) kW")
    for t in 1:hours_per_year
        production_by_tech[g] += value(p[g, t])  # Accumulate as floating-point values
        total_production += value(p[g, t])  # Keep total production as floating-point
    end
end

println("\n")

println("Point II")
println("Share of each technology in effective production:")
for g in keys(technologies)
    share = production_by_tech[g] / total_production
    println("$(g): $(share * 100) %")
end
println("\n")


println("Point III")
println("Total Production: $(total_production) MWh")
total_cost = objective_value(model)
println("Total Investment and Production Cost: €$(total_cost)")
println("\n")


println("Point IV")
# CO2 Emissions data
emissions = Dict(
    "Coal" => 1.4,
    "CCGT" => 0.5,
    "OCGT" => 0.6,
    "Onshore Wind" => 0.0,
    "Offshore Wind" => 0.0,
    "PV" => 0.0
)

# Calculating CO2 emissionys
total_co2_emissions = sum(emissions[g] * production_by_tech[g] for g in keys(production_by_tech))
println("\nTotal CO2 Emissions: $(total_co2_emissions) tons")
println("\n")


println("Point V")
load_curtailment = sum(max(0, D_t[t] - sum(value(p[g, t]) for g in keys(technologies))) for t in 1:hours_per_year)
println("Total Load Curtailment: $(load_curtailment) MWh")
println("\n")


println("Point VI")
# Profit calculation per technology
profit_by_tech = Dict()
for g in keys(technologies)
    revenue = lambda * production_by_tech[g]  # Assuming all production is sold at threshold price λ
    cost = technologies[g].capex * value(k[g]) + technologies[g].om * technologies[g].capex * value(k[g]) / economic_life
    profit_by_tech[g] = revenue - cost
end

println("\nProfit by Technology:")
for g in keys(profit_by_tech)
    println("$(g): €$(profit_by_tech[g])")
end

Running HiGHS 1.5.3 [date: 1970-01-01, git hash: 45a127b78]
Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
57214 rows, 57220 cols, 154122 nonzeros
57214 rows, 57220 cols, 154122 nonzeros
Presolve : Reductions: rows 57214(-12866); columns 57220(-4106); elements 154122(-16972)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Ph1: 0(0) 0s
      46711    -2.3600998855e+12 Pr: 0(0) 1s
      46711    -2.3600998855e+12 Pr: 0(0) 1s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 46711
Objective value     : -2.3600998855e+12
HiGHS run time      :          1.29


Point I
Optimal capacities (kW):
Coal: 0.0 kW
CCGT: 42819.5652173913 kW
Offshore Wind: 0.0 kW
OCGT: 52030.4347826087 kW
PV: 815.2173913043953 kW
Onshore Wind: 0.0 kW


Point II
Share of each technology in effective production:
Coal: 0.0 %
CCGT: 77.90093